<a href="https://colab.research.google.com/github/ramahasiba/Free_LLM_Deployment/blob/main/Ollama_Deployment_Using_Ngrok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Required Dependecies

### Install pyngrok and openai

In [ ]:
!pip install pyngrok==7.2.0 -q
!pip install openai -q

### Download and setup Ollama on this Google colab instance

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


## Configure Model Selection

### Define the Ollama model to use

In [ ]:
ollama_model_id = "gemma2:9b-instruct-q5_0"

In [ ]:
# Kill any existing Ollama processes to prevent conflicts (uncomment the line below to )

# !pkill -f ollama

## Start the Ollama server

In [ ]:
# Run the process in the background, and allow cross-origin requests from any domain
!nohup bash -c "OLLAMA_HOST=0.0.0.0:8000 OLLAMA_ORIGIN=* ollama serve" &

# Wait 5 seconds for the server to start, then display the server logs
# This helps verify that the server started successfully
!sleep 5 && tail /content/nohup.out

nohup: appending output to 'nohup.out'

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAILmEnm1/IZpMWF0rn8/K7oDDrrd3aJ9akuEFNjBr6003

time=2025-10-20T13:46:59.363Z level=INFO source=routes.go:1511 msg="server config" env="map[CUDA_VISIBLE_DEVICES: GGML_VK_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_CONTEXT_LENGTH:4096 OLLAMA_DEBUG:INFO OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://0.0.0.0:8000 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_KV_CACHE_TYPE: OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/root/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NEW_ENGINE:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:1 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0

## Download the Selected Model

In [ ]:
!OLLAMA_HOST=0.0.0.0:8000 ollama pull {ollama_model_id}

## Checking if the Model Pulled Correctly

### Test the model locally by sending a chat request to the Ollama API, this verifies that the model is working before exposing it via ngrok

In [ ]:
%%bash
curl http://localhost:8000/api/chat -d '{
  "model": "gemma2:9b-instruct-q5_0",
  "stream": false,
  "messages": [
    { "role": "user", "content": "what is AI?" }
  ]
}'

{"model":"gemma2:9b-instruct-q5_0","created_at":"2025-10-20T12:16:59.789188693Z","message":{"role":"assistant","content":"AI, or Artificial Intelligence, is a broad field of computer science focused on creating machines capable of performing tasks that typically require human intelligence.\n\nHere's a breakdown:\n\n**What AI can do:**\n\n* **Learn from data:** AI algorithms can analyze vast amounts of data and identify patterns, trends, and insights that humans might miss.\n* **Make decisions:** Based on the data they learn, AI systems can make predictions and recommendations, even in complex situations.\n* **Understand natural language:** Some AI models can understand and respond to human language, allowing for interactions like chatbots and virtual assistants.\n* **Solve problems:** AI can be used to solve problems in various fields, such as medicine, finance, and transportation.\n* **Create content:** AI can generate text, images, music, and even code.\n\n**Types of AI:**\n\n* **Nar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2896    0  2767  100   129     92      4  0:00:32  0:00:29  0:00:03   747


## Setup Ngrok

In [ ]:
from google.colab import userdata
from pyngrok import ngrok, conf

In [ ]:
# Retrieve the ngrok authentication token from Colab secrets
# You need to add your ngrok token to Colab secrets with the key 'colab-ngrok'
# Get your token from: https://dashboard.ngrok.com/get-started/your-authtoken

ngrok_auth = userdata.get('colab-ngrok')

In [ ]:
# Configure ngrok with the authentication token
conf.get_default().auth_token = ngrok_auth

In [ ]:
# Specify the local port where Ollama is running
port = "8000"

In [ ]:
# Create a public ngrok tunnel to the local Ollama server
public_url = ngrok.connect(port).public_url
print(public_url)

https://lavina-hamulate-crankly.ngrok-free.dev


## Check if the model works after deployment


In [ ]:
%%bash
curl https://lavina-hamulate-crankly.ngrok-free.dev/api/chat -d '{
  "model": "gemma2:9b-instruct-q5_0",
  "stream": false,
  "messages": [
    { "role": "user", "content": "How are you?" }
  ]
}'

{"model":"gemma2:9b-instruct-q5_0","created_at":"2025-10-20T12:17:27.3328715Z","message":{"role":"assistant","content":"As an AI, I don't have feelings or experiences like humans do. However, I'm here and ready to assist you with any questions or tasks you may have.\n\nIs there anything specific I can help you with today?"},"done":true,"done_reason":"stop","total_duration":2298203625,"load_duration":234926263,"prompt_eval_count":13,"prompt_eval_duration":151645343,"eval_count":49,"eval_duration":1657806860}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   642  100   512  100   130    202     51  0:00:02  0:00:02 --:--:--   253


## Python Client Example

In [1]:
import os
from openai import OpenAI

# Get the ngrok public URL from the output of cell KFRnCctX1JFG
# Replace the placeholder with your actual ngrok URL
# You can find the URL printed in the output of the ngrok connection cell
ngrok_url = "https://lavina-hamulate-crankly.ngrok-free.dev" # Replace with your ngrok URL

# Configure the OpenAI client to use the local Ollama instance via ngrok
# The API key is not required by Ollama, but the library expects one.
client = OpenAI(
    base_url = f"{ngrok_url}/v1",
    api_key = "dummy key"
)

## Generate a Chat Completion


In [ ]:
# Send a chat completion request to the deployed model
try:
    response = client.chat.completions.create(
        model=ollama_model_id,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Tell me a short story."}
        ],
        max_tokens=400,
        stream=False
    )

    # Extract and print the model's response
    print(response.choices[0].message.content)

except Exception as e:
    print(f"An error occurred: {e}")